In [1]:
!pip install pytorch-lightning --upgrade

!pip install torchmetrics==0.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.8/661.8 KB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 KB 25.3 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 KB 18.2 MB/s eta 0:00:00
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 0.11.4
    Uninstalling torchmetrics-0.11.4:
      Successfully uninstalled torchmetrics-0.11.4
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [4]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
#from pytorch_lightning import metrics
from sklearn.preprocessing import LabelEncoder
#from sklearn.model_selection import train_test_splitb

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
comp_df = pd.read_csv('input/labels.csv')
test_df = pd.read_csv('input/sample_submission.csv')

print('Training set: {}, Test set: {}'.format(comp_df.shape[0],test_df.shape[0]))

FileNotFoundError: [Errno 2] No such file or directory: 'input/labels.csv'

In [7]:
comp_df.breed.value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
komondor                 67
golden_retriever         67
brabancon_griffon        67
briard                   66
eskimo_dog               66
Name: breed, Length: 120, dtype: int64

In [8]:
comp_df['label'] = LabelEncoder().fit_transform(comp_df.breed)

# Create a breed-2-index dictionary
dict_df = comp_df[['label','breed']].copy()
dict_df.drop_duplicates(inplace=True)
dict_df.set_index('label',drop=True,inplace=True)
index_to_breed = dict_df.to_dict()['breed']

In [9]:
train_dir = '../input/dog-breed-identification/train'
comp_df.id = comp_df.id.apply(lambda x: x+'.jpg')
comp_df.id = comp_df.id.apply(lambda x:train_dir+'/'+x)

comp_df.pop('breed')

0                     boston_bull
1                           dingo
2                        pekinese
3                        bluetick
4                golden_retriever
                   ...           
10217                      borzoi
10218              dandie_dinmont
10219                    airedale
10220          miniature_pinscher
10221    chesapeake_bay_retriever
Name: breed, Length: 10222, dtype: object

In [10]:
def show_images(df,img_num):
    sample = df.sample(img_num)
    paths = sample.id.tolist()
    for path in paths:
        plt.figure(figsize=(3,3))
        img = plt.imread(path)
        plt.imshow(img)
        plt.axis('off')
        plt.show()


In [11]:
show_images(comp_df,4)

FileNotFoundError: [Errno 2] No such file or directory: 'train/0ca9280205bd68d201e1d708db3f209d.jpg'

<Figure size 300x300 with 0 Axes>

In [ ]:
class img_dataset(Dataset):
    def __init__(self,dataframe,transform=None,test=False):
        self.dataframe = dataframe
        self.transform = transform
        self.test = test
        
    def __getitem__(self,index):
        x = Image.open(self.dataframe.iloc[index,0])
        if self.transform:
            x = self.transform(x)
        if self.test:
            return x
        else:
            y = self.dataframe.iloc[index,1]
            return x,y
        
    def __len__(self):
        return self.dataframe.shape[0]

In [ ]:
train_transformer = transforms.Compose([transforms.RandomResizedCrop(224),
                                        transforms.RandomRotation(15),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

val_transformer = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [ ]:
training_samples = comp_df.shape[0] 
test_size=0.05
batch_size = 64

sample_df = comp_df.sample(training_samples)

x_train,x_val,_,_ = train_test_split(sample_df,sample_df,test_size=test_size)

train_set = img_dataset(x_train, transform=train_transformer)
val_set = img_dataset(x_val, transform=val_transformer)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set , batch_size=batch_size, shuffle=True)

print('Training set: {}, Validation set: {}'.format(x_train.shape[0],x_val.shape[0]))

In [ ]:
class net(torch.nn.Module):
    def __init__(self, base_model, base_out_features, num_classes):
        super(net,self).__init__()
        self.base_model=base_model
        self.linear1 = torch.nn.Linear(base_out_features, 512)
        self.output = torch.nn.Linear(512,num_classes)
    def forward(self,x):
        x = F.relu(self.base_model(x))
        x = F.relu(self.linear1(x))
        x = self.output(x)
        return x

res = torchvision.models.resnet50(pretrained=True)
for param in res.parameters():
    param.requires_grad=False

model_final = net(base_model=res, base_out_features=res.fc.out_features, num_classes=120)
model_final = model_final.to(device)

In [ ]:
cost_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([param for param in model_final.parameters() if param.requires_grad], lr=0.0003)

EPOCHS = 30

In [ ]:
train_losses, train_acc, val_losses, val_acc = train_model(model=model_final, 
                                                           cost_function=cost_function, 
                                                           optimizer=optimizer,
                                                           num_epochs=EPOCHS

In [ ]:
def plot_result(train_loss,val_loss,train_acc,val_acc):
    plt.style.use('ggplot')
    fig, (ax1,ax2) = plt.subplots(2,1,figsize=(10,8))
    ax1.plot(train_loss,label='loss')
    ax1.plot(val_loss,label='val_loss')
    ax1.legend()
    ax1.set_xlabel('epoch')
    ax1.set_xticks(range(0,EPOCHS+1))
    ax2.plot(train_acc, label='acc')
    ax2.plot(val_acc,label='val_acc')
    ax2.legend()
    ax2.set_xlabel('epoch')
    ax2.set_xticks(range(0,EPOCHS+1))
    plt.show()
plot_result(train_losses,val_losses, train_acc,  val_acc)

In [ ]:
test_dir = '../input/dog-breed-identification/test'
test_df = test_df[['id']]
test_df.id = test_df.id.apply(lambda x: x+'.jpg')
test_df.id = test_df.id.apply(lambda x : test_dir+'/'+x)
test_set = img_dataset(test_df,transform=val_transformer, test=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
model_final.eval()
predictions = torch.tensor([])
print('Start predicting....')
for x in test_loader:
    x = x.to(device)
    y_hat = model_final(x)
    predictions = torch.cat([predictions, y_hat.cpu()])
print('Finish prediction.')

In [ ]:
predictions = F.softmax(predictions,dim=1).detach().numpy()

In [ ]:
answer_id = pd.read_csv('../input/dog-breed-identification/sample_submission.csv').id.tolist()
predictions_df = pd.DataFrame(predictions, index=answer_id)
predictions_df.columns = predictions_df.columns.map(index_to_breed)
predictions_df.rename_axis('id', inplace=True)
predictions_df.to_csv('submission.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=99a26ad6-4629-4ddb-a331-c924192d30a8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>